In [1]:
import orjson

with open("../data/mptraj.json", "rb") as f:   # 注意 orjson 用二进制模式
    data = orjson.loads(f.read())

In [24]:
# import os 
# os.environ["HF_ENDPOINT"] = 'https://hf-mirror.com'


from datasets import load_dataset,load_from_disk

ds = load_dataset("/root/.cache/huggingface/hub/datasets--nimashoghi--mptrj/snapshots/f88fbe46e16524223210654bad9e1b05a15c2adb/data/")

In [26]:
ds

DatasetDict({
    train: Dataset({
        features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 'ionic_step', 'filename', 'extxyz_id', 'num_atoms', 'corrected_total_energy_relaxed', 'energy_referenced', 'corrected_total_energy_referenced', 'corrected_total_energy_relaxed_referenced', 'composition'],
        num_rows: 1559916
    })
    validation: Dataset({
        features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 'ionic_step', 'filename', 'extxyz_id', 'num_atoms', 'corrected_total_energy_relaxed', 'energy_referenced', 'corrected_total_energy_referenced', 'corrected_total_energy_relaxed_referenced', 'composition'],
        num_rows: 10206
    })
    test: Dataset

In [29]:
import lmdb
import pickle
from tqdm import tqdm
def save_lmdb(lmdb_path = '',atoms_list=None):
    
    env = lmdb.open(
        lmdb_path,
        map_size=1 << 40,  # 1 TB
        subdir=False,
        meminit=False,
        map_async=True
    )

    with env.begin(write=True) as txn:
        for idx, atoms in tqdm(enumerate(atoms_list), total=len(atoms_list)):
            txn.put(
                key=f"{idx}".encode("ascii"),
                value=pickle.dumps(atoms)
            )
        txn.put(b'length', pickle.dumps(len(atoms_list)))
        print(f"✅ 已保存 {len(atoms_list)} 个结构到 {lmdb_path}")
    env.sync()
    env.close()

save_lmdb(lmdb_path='/ve/junfu/AtomScaffold2/data/mptraj/train/data',atoms_list=ds['train'])

100%|██████████| 1559916/1559916 [18:09<00:00, 1431.76it/s]


✅ 已保存 1559916 个结构到 /ve/junfu/AtomScaffold2/data/mptraj/train/data


In [35]:
from asf_datasets.lmdb_data import LMDBDataset
train_set = LMDBDataset(lmdb_path='/ve/junfu/AtomScaffold2/data/mptraj',split='train',label_keys=['energy_per_atom','forces','stress'])

read length:1559916
available keys:dict_keys(['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 'ionic_step', 'filename', 'extxyz_id', 'num_atoms', 'corrected_total_energy_relaxed', 'energy_referenced', 'corrected_total_energy_referenced', 'corrected_total_energy_relaxed_referenced', 'composition'])
used keys:['energy_per_atom', 'forces', 'stress']


In [36]:
train_set[0]

Data(x=[14], pos=[14, 3], cell=[3, 3], atomic_numbers=[14], natoms=14, energy_per_atom=-6.793888092041016, forces=[14, 3], stress=[3, 3])